## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-28-16-52-22 +0000,nyt,"A Gaza City Neighborhood Is Now a Wasteland, S...",https://www.nytimes.com/2025/08/28/world/middl...
1,2025-08-28-16-51-25 +0000,nyt,"New York Needs More Millionaires, Fiscal Watch...",https://www.nytimes.com/2025/08/28/nyregion/ny...
2,2025-08-28-16-51-00 +0000,wsj,Central U.S. Manufacturing Maintains Expansion...,https://www.wsj.com/economy/central-u-s-manufa...
3,2025-08-28-16-48-19 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/08/28/us/min...
4,2025-08-28-16-44-15 +0000,bbc,"Electric cars eligible for £3,750 discount ann...",https://www.bbc.com/news/articles/cvgvywyev1do...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2295/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
77,trump,52
145,minneapolis,19
146,school,17
88,shooting,15
194,fire,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
23,2025-08-28-16-12-00 +0000,wsj,"Fed governor Lisa Cook sued President Trump, s...",https://www.wsj.com/economy/central-banking/li...,148
206,2025-08-28-01-00-00 +0000,wsj,"In his first term, administration officials re...",https://www.wsj.com/politics/policy/in-trumps-...,140
69,2025-08-28-14-18-31 +0000,cbc,U.S. Fed governor Lisa Cook suing Trump admini...,https://www.cbc.ca/news/world/trump-lisa-cook-...,121
44,2025-08-28-15-24-39 +0000,bbc,US Fed Governor Lisa Cook sues Trump over his ...,https://www.bbc.com/news/articles/c1dxl6ry4y3o...,114
48,2025-08-28-15-14-52 +0000,bbc,Can Trump fire Fed Governor Lisa Cook and why ...,https://www.bbc.com/news/articles/cedvj2d5538o...,107


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
23,148,2025-08-28-16-12-00 +0000,wsj,"Fed governor Lisa Cook sued President Trump, s...",https://www.wsj.com/economy/central-banking/li...
12,87,2025-08-28-16-31-00 +0000,wsj,Susan Monarez had two lines she wouldn’t cross...,https://www.wsj.com/politics/policy/cdc-direct...
101,74,2025-08-28-11-59-49 +0000,nypost,Minneapolis school shooter Robin Westman confe...,https://nypost.com/2025/08/28/us-news/minneapo...
184,69,2025-08-28-03-19-08 +0000,nypost,Russian ballistic missiles hit Kyiv in large-s...,https://nypost.com/2025/08/27/world-news/russi...
180,40,2025-08-28-04-19-34 +0000,nypost,Jared Kushner and Tony Blair join Trump for Ga...,https://nypost.com/2025/08/28/us-news/jared-ku...
11,37,2025-08-28-16-34-11 +0000,startribune,Live: One child still critical at HCMC after d...,https://www.startribune.com/robin-westman-annu...
311,34,2025-08-27-17-56-38 +0000,latimes,"Cal Fire captain fatally shot his girlfriend, ...",https://www.latimes.com/california/story/2025-...
46,34,2025-08-28-15-22-15 +0000,nypost,Anti-Israel protesters rally outside Venice Fi...,https://nypost.com/2025/08/28/world-news/anti-...
308,34,2025-08-27-18-21-00 +0000,wsj,President Trump’s tariff rollout has taken num...,https://www.wsj.com/economy/trade/trump-tariff...
157,33,2025-08-28-08-02-39 +0000,nypost,"Vladimir Putin, Kim Jong Un to attend Chinese ...",https://nypost.com/2025/08/28/world-news/putin...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
